In [1]:
import numpy as np
from cloudant import cloudant
from ipywidgets import interact
from bokeh.charts import Histogram
from bokeh.io import push_notebook, show, output_notebook, push_notebook
from bokeh.layouts import gridplot
from bokeh.models import Span, Label, ColumnDataSource, HoverTool
from bokeh.plotting import figure

In [2]:
# The code was removed by DSX for sharing.

In [3]:
#### Pull aggregate data from cloudant
aggregate = None
latest_replay = None
with cloudant(sc2replay_creds["user"], sc2replay_creds["password"], account="e-sports") as esports:
    session = esports.session()
    sc2db = esports["sc2aggregate"]
    
    for aggs in sc2db:
        aggregate = aggs
        
    sc2latest = esports["sc2recents"]
    for recent in sc2latest:
        latest_replay = recent
        

In [4]:
#### Pull latest replay from 

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
def parse_players(replay_data):
    players = {}

    for player in replay_data["winners"]:
        players[player[0]] = {"full": "Winning Player {num}: {name} ({race})".format(num=player[0], name=player[1], race=player[2]),
                              "short": "{name} ({race})".format(name=player[1], race=player[2]),
                              "race": player[2],
                              "winner": True, 
                              "num": player[0]}
    for player in replay_data["losers"]:
        players[player[0]] = {"full": "Losing Player {num}: {name} ({race})".format(num=player[0], name=player[1], race=player[2]),
                              "short": "{name} ({race})".format(name=player[1], race=player[2]),
                              "race": player[2],
                              "winner": False,
                              "num": player[0]}
    return players

def get_percentile(percentiles, mean, stat):
    if mean is not None:
        if stat in ["third_vespene_structure", "second_base"]:
            mean = mean /16
        group = [ndx for ndx, k in enumerate(percentiles) if k<=mean]
        if len(group) == 0:
            return 0
        return max(group)
    return "N/A"


def all_histogram(stat, players, aggregate, replay):
    span = []
    label = []
    perc_bins = aggregate["basic"][stat]["all"]["percentiles"]
    player_titles = []
    mean = aggregate["basic"][stat]["all"]["mean"]
    
    
    player_mean = {}
    for player, player_info in players.items():
        my_mean = 0
        
        if stat == "second_base":
            expansion_events = replay["stats"][str(player)].get("expansion_event")
            if expansion_events:
                if len(expansion_events) > 0:
                    plus_counts = 0
                    for i, event in enumerate(expansion_events):
                        if event[1] == "+":
                            plus_counts += 1
                        if plus_counts == 2:
                            my_mean = (replay["stats"][str(player)]["expansion_event"][i][0])/16
                my_percentile = 1 - get_percentile(perc_bins, my_mean, stat)
            else:
                my_percentile = "N/A"
        elif stat == "third_vespene_structure":
            vespene_events = replay["stats"][str(player)].get("vespene_event")
            if vespene_events:
                if len(vespene_events) >= 3:
                    plus_counts = 0
                    for i, event in enumerate(vespene_events):
                        if event[1] == "+":
                            plus_counts += 1
                        if plus_counts == 3:

                            my_mean = (replay["stats"][str(player)]["vespene_event"][i][0])/16
                            break
                my_percentile = 100 - get_percentile(perc_bins, my_mean, stat) 
            else:
                my_percentile = "N/A"
        else:
            my_percentile = "N/A"
            if replay["stats"][str(player)].get(stat, None):
                my_mean = np.mean([point[1] for point in replay["stats"][str(player)][stat]])
            else:
                my_percentile = "N/A"
        
            my_percentile = get_percentile(perc_bins, my_mean, stat)
            
        player_titles.append("{0}-{1} Percentile: {2}".format("Winner" if player_info["winner"] else "Loser", 
                                                              player_info["short"],
                                                              my_percentile))
        player_mean[player] = my_mean
    percentile_title = ", ".join(player_titles)
    
                                        
    p_all = Histogram(aggregate["basic"][stat]["all"]["points"], 
                      title="{0}: {1}  Mean: {2:.4f},  {3}".format(stat, "all", mean, percentile_title), 
                      bins=40,
                      density=True,
                      plot_width=1040,
                      plot_height=200,
                      toolbar_location=None)
    p_all.yaxis.visible = False
    p_all.xaxis.axis_label = stat
    
    mean_span = Span(location=mean,
                     dimension="height",
                     line_color="black",
                     line_width=4)
    
    p_all.add_layout(mean_span)
    
    for player, player_info in players.items():
        mean = 0
        span.append(Span(location=player_mean[player],
                         dimension="height",
                         line_color="green" if player_info["winner"] else "blue",
                         line_width=3))

        label.append(Label(x=1.15 * player_mean[player], 
                           y=.02 if player==1 else .03, 
                           text="{0}: {1:.04f}".format(player_info["short"], 
                                                       mean)))

    for ndx in range(0,len(label)):
        p_all.add_layout(span[ndx])
        p_all.add_layout(label[ndx])
    
    return p_all

def player_histogram(stat, player, aggregate, replay):
    span = None
    label = None
    
    p_mean = None
    percentile = None
    if stat == "second_base":
        print("second_base")
        expansion_events = replay["stats"][str(player["num"])].get("expansion_event")
        if expansion_events:
            if len(expansion_events) > 0:
                plus_counts = 0
                for i, event in enumerate(expansion_events):
                    if event[1] == "+":
                        plus_counts += 1
                    if plus_counts == 2:
                        p_mean = (replay["stats"][str(player["num"])]["expansion_event"][i][0])/16
                        break
            else:
                p_mean = 0

            percentile = 100 - get_percentile(aggregate["basic"][stat][player["race"].lower()]["percentiles"], p_mean, stat)
    elif stat == "third_vespene_structure":
        print("third_vespene_structure", stat)
        vespene_events = replay["stats"][str(player["num"])].get("vespene_event")
        if vespene_events:
            if len(vespene_events) >= 3:
                plus_counts = 0
                for i, event in enumerate(vespene_events):
                    if event[1] == "+":
                        plus_counts += 1
                    if plus_counts == 3:
                        p_mean = (replay["stats"][str(player["num"])]["vespene_event"][i][0])/16
                        break
            else:
                p_mean = 0
        
            percentile = 100 - get_percentile(aggregate["basic"][stat][player["race"].lower()]["percentiles"], p_mean, stat)
        
    else:
        if  replay["stats"][str(player["num"])].get(stat):
            p_mean = np.mean([point[1] for point in replay["stats"][str(player["num"])][stat]])
        else:
            p_mean = 0
    
        percentile = get_percentile(aggregate["basic"][stat][player["race"].lower()]["percentiles"], p_mean, stat)
    mean = aggregate["basic"][stat][player["race"].lower()]["mean"]
    
    print(player["short"], player["race"], p_mean, mean/16, percentile)
    if stat in ["third_vespene_structure", "second_base"]:
        p_player = Histogram([p/16 for p in aggregate["basic"][stat][player["race"].lower()]["points"]],
                             title="{0}:  {1}  Value:{2:.4f} Mean:{3:.4f}  Percentile:{4}".format(player["short"], 
                                                                                                  player["race"].lower(), 
                                                                                                  p_mean/16, 
                                                                                                  mean/16, 
                                                                                                  percentile),
                             bins=40,
                             density=True,
                             plot_width=500,
                             plot_height=200,
                             toolbar_location=None)

        mean_span = Span(location=aggregate["basic"][stat][player["race"].lower()]["mean"]/16,
                         dimension="height",
                         line_color="black",
                         line_width=4)
    else:
            
        p_player = Histogram(aggregate["basic"][stat][player["race"].lower()]["points"],
                             title="{0}:  {1}  Value:{2:.4f} Mean:{3:.4f}  Percentile:{4}".format(player["short"], 
                                                                                                  player["race"].lower(), 
                                                                                                  p_mean, 
                                                                                                  mean, 
                                                                                                  percentile),
                             bins=40,
                             density=True,
                             plot_width=500,
                             plot_height=200,
                             toolbar_location=None)

        mean_span = Span(location=aggregate["basic"][stat][player["race"].lower()]["mean"],
                         dimension="height",
                         line_color="black",
                         line_width=4)
    
    p_player.add_layout(mean_span)
    
    p_player.yaxis.visible=False
    p_player.xaxis.axis_label = stat
    
    if p_mean is not None:
        span = Span(location=p_mean,
                    dimension="height",
                    line_color="green" if player["winner"] else "blue",
                    line_width=3)

        label = Label(x=1.15 * p_mean, 
                      y=.02 if player["num"]==1 else .03, 
                      text="{0:.04f}".format(mean))

        p_player.add_layout(span)
        p_player.add_layout(label)
    
    return p_player




In [7]:
basic_stats = aggregate["stats"]["basic"]
players = parse_players(latest_replay)
grid = []
for s in basic_stats:
    p_all = all_histogram(s, players, aggregate, latest_replay)
    player_row = [player_histogram(s, player, aggregate, latest_replay) for _, player in players.items()] 
    grid.extend([[p_all], player_row])


Namshar (Zerg) Zerg 50.4855491329 2.213690098437186 88
Impact (Zerg) Zerg 47.9022988506 2.213690098437186 82
Namshar (Zerg) Zerg 0.474020663281 0.04072367798463248 2
Impact (Zerg) Zerg 0.478108883421 0.04072367798463248 2
Namshar (Zerg) Zerg 0.844782918571 0.05231079295381855 51
Impact (Zerg) Zerg 0.834418407493 0.05231079295381855 41
Namshar (Zerg) Zerg 1776.13872832 93.1598347889395 69
Impact (Zerg) Zerg 1733.62068966 93.1598347889395 65
Namshar (Zerg) Zerg 36.8537008877 2.7912856376392563 2
Impact (Zerg) Zerg 37.297222712 2.7912856376392563 3
Namshar (Zerg) Zerg 739.445086705 21.54871547411995 97
Impact (Zerg) Zerg 636.408045977 21.54871547411995 91
Namshar (Zerg) Zerg 12.5238874049 0.4742754855409642 96
Impact (Zerg) Zerg 11.8188889299 0.4742754855409642 94
second_base
Namshar (Zerg) Zerg 248.625 191.37017886273597 100
second_base
Impact (Zerg) Zerg 341.875 191.37017886273597 100
third_vespene_structure third_vespene_structure
Namshar (Zerg) Zerg 379.875 488.78363031508104 100
thir

In [8]:
show(gridplot(grid, sizing_mode="scale_width"), notebook_handle=True)